As an additional exploration with the species identified as having GAP habitat maps available, this notebook explores adding a view of GAP habitat metrics to the cache of information. This may be useful in determining where else species may have range within states. The process uses the 2018 US State boundaries from Census, the GAP range bounding box (generated previously in the basic caching of GAP species data from the GAP range services), and a set of pre-calculated GAP metrics that include acres within specific protection status designations in each state. It first finds a basic intersection of the range bounding box with a set of states, retrieves the total state metrics for all GAP species, and then filters to an individual species (no current query criteria for this at the API). This information is then cached for further reference.

In [1]:
import geopandas as gpd
import json
import requests
from IPython.display import display
import bispy
from joblib import Parallel, delayed
import jsonschema

gap = bispy.gap.Gap()
bis_utils = bispy.bis.Utils()

import helperfunctions

In [2]:
us_states = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2018/STATE/tl_2018_us_state.zip")

In [3]:
# Open the file back up and verify
with open("../cache/gap.json", "r") as f:
    gap_cache = json.loads(f.read())
    f.close()

In [5]:
# Tease out the unique GAP species that we were able to find in the entire set of searched names
gap_species = list(map(json.loads,set(map(json.dumps, [r for r in gap_cache if "data" in r.keys()]))))

In [7]:
%%time
# Use joblib to run multiple requests for SGCN records in parallel via scientific names
gap_metrics = Parallel(n_jobs=8)(delayed(gap.gap_metrics_species)(us_states, spp["data"]["GAP_SpeciesCode"], spp["data"]["Range Bounding Box"]) for spp in gap_species)


CPU times: user 4.13 s, sys: 1.95 s, total: 6.08 s
Wall time: 3min 15s


In [8]:
# Cache the array of retrieved documents and return/display a random sample for verification
display(bis_utils.doc_cache("../cache/gap_metrics.json", gap_metrics))

{'Doc Cache File': '../cache/gap_metrics.json',
 'Number of Documents in Cache': 65,
 'Document Number 14': {'GAP_SpeciesCode': 'aGBSAx',
  'State Metrics': [{'state_fipscode': '01',
    'state_name': 'Alabama',
    'taxa': 'A',
    'sppcode': 'aGBSAx',
    'spp_comname': 'Georgia Blind Salamander',
    'spp_sciname': 'Haideotriton wallacei',
    'gapstat1ac': 0.0,
    'gapstat2ac': 0.8895793716,
    'gapstat3ac': 0.0,
    'gapstat4ac': 3441.7825887204,
    'gapstat12ac': 0.8895793716,
    'gapstat123ac': 0.8895793716,
    'totalac': 3442.672168092,
    'gapstat1perc': 0.0,
    'gapstat2perc': 0.0258397932816537,
    'gapstat3perc': 0.0,
    'gapstat4perc': 99.9741602067183,
    'gapstat12perc': 0.0258397932816537,
    'gapstat123perc': 0.0258397932816537,
    'gapstat12group': '<1',
    'gapstat123group': '<1',
    'id': 1745},
   {'state_fipscode': '12',
    'state_name': 'Florida',
    'taxa': 'A',
    'sppcode': 'aGBSAx',
    'spp_comname': 'Georgia Blind Salamander',
    'spp_scin

# Schema Validation

In [10]:
gap_metrics_schema = helperfunctions.load_schema('gap_metrics')
display(gap_metrics_schema)

jsonschema.validate(gap_metrics, gap_metrics_schema)

{'definitions': {'items': {'$id': '#items',
   'type': ['object', 'array'],
   'title': 'Generic container for items in a dataset',
   'description': 'A JSON array or object property containing one or more items in a dataset or data structure within a dataset.'}},
 '$schema': 'http://json-schema.org/draft-07/schema#',
 '$id': 'http://data.usgs.gov/property_registry/',
 'type': 'array',
 'title': 'Cache of GAP Metrics for US States',
 'description': 'This dataset contains a set of pre-calculated protection status metrics pulled from an API for a collection of GAP species. Metrics are for US states, showing the calculated acres and percentage land cover with from the 3-meter GAP species habitat maps.',
 'required': ['GAP_SpeciesCode', 'State Metrics'],
 'properties': {'GAP_SpeciesCode': {'$id': '#/properties/GAP_SpeciesCode',
   'type': 'string',
   'title': 'GAP Species Code',
   'description': 'The unique, persistent identifier for a GAP species.',
   'examples': ['aNRWAx']},
  'State 